In [41]:
import pandas as pd
import numpy as np
import time

from collections import Counter
import requests
import wikipediaapi as wiki

import spacy

# train and test are the same dataset
train = pd.read_csv('data/train.csv')
# leaving test out cause its same as the train data without answer column
# test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [42]:
train.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [43]:
test.head()

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [44]:
train.iloc[0][1]

'Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?'

In [45]:
'''
Main process was understanding the word frequency, from frequency see what type of words need
to be kept or filterered out. Looked like lot of keywords were PROPN: Proper Noun; names, etc.
But some questions had no PROPN, then for those questions, looked like Nouns; place, things, etc.
were good to extract

NOTE: There are still two questions with no keywords, come back to this later
'''
#tagged_words = train['token_tag'].explode().tolist()
#tagged_words = pd.DataFrame(tagged_words, columns = ['word', 'tag'])

# for i in range(10):
#     display(train['prompt'][i])
#     display(train['token_tag'].apply(filter_tag)[i])

# train['prompt'][2]
# tagged_words.loc[tagged_words['word']=='scaling']

# word tagger for sentences
def spacy_tagger(sentence):
    doc = spacyyy(sentence)
    tagged_words = [(token.text, token.pos_) for token in doc]
    
    return tagged_words

def filter_tag(list_of_tuples):
    return [word for word, tag in list_of_tuples if tag in ['PROPN', 'NOUN']]

spacyyy = spacy.load('en_core_web_sm')
train['token_tag'] = train['prompt'].apply(spacy_tagger)

train['propn_noun'] = train['token_tag'].apply(filter_tag)

In [46]:
train.head()

,id,prompt,A,B,C,D,E,answer,token_tag,propn_noun
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, impact, Modified, Newtonian, Dyna..."
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[following, definition, scaling, self, systems]"
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, origin, significance, triskeles, ..."
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,"[(What, PRON), (is, AUX), (the, DET), (signifi...","[significance, regularization, terms, renormal..."
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, relationship, dimensions, object,..."


In [94]:
# searches wiki for relevant article
def search_wiki(search_term):
    print(search_term)
    search_url = 'https://en.wikipedia.org/w/api.php'
    search_params = {
        'action': 'query',
        'format': 'json',
        'list': 'search',
        'srsearch': search_term
    }
    
    response = requests.get(search_url, params = search_params)
    if response.status_code == 200:
        results = response.json()['query']['search']
        if len(results) > 0:
            # cause I just want the top result
            return results[0]['title']
        else:
            return None
    else:
        return None

# collects article based on page title
def get_wiki_article(page_title, user_agent='WikiForScienceExam'):
    wiki_wiki = wiki.Wikipedia(user_agent)
    
    page = wiki_wiki.page(page_title)
    
    if page.exists():
        return page.text
    else:
        return None

# function to support addition of articles to dataframe based on keywords
def keyword_based_articles(keywords):
    article = None
    
    topic = ' '.join(keywords)
    if topic.strip() == '':
        return article 
    
    top_title = search_wiki(topic)
    
    if top_title:
        article = get_wiki_article(top_title)
    
    time.sleep(1)
    return article

train['wiki_article'] = train['propn_noun'].apply(keyword_based_articles)

statements impact Modified Newtonian Dynamics MOND mass discrepancy galaxy clusters
following definition scaling self systems
statements origin significance triskeles symbol
significance regularization terms renormalization problems physics
statements relationship dimensions object spacing features diffraction pattern
statements relationship Gauss law flux field symmetry fields
statements dimension object CW complex
statements temperature layer spin valve
term astrophysics matter interactions energy shifts radiation field
role axioms theory
Fresnel regards reflections
relationship Wigner function density matrix operator
examples models cosmologists physicists principles issues Lambda CDM model models models
Roche limit
Martin Heidegger view relationship time existence
catastrophe
explanation shower curtain effect
butterfly effect
effect materials
length inverse length
statements categorization systems dynamics
propagation waves
interaction
Newton manuscripts show
decay energy neutron d

In [95]:
train.head()

,id,prompt,A,B,C,D,E,answer,token_tag,propn_noun,wiki_article
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, impact, Modified, Newtonian, Dyna...",The rotation curve of a disc galaxy (also call...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[following, definition, scaling, self, systems]","Self-organization, also called spontaneous ord..."
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, origin, significance, triskeles, ...",A triskelion or triskeles is an ancient motif ...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C,"[(What, PRON), (is, AUX), (the, DET), (signifi...","[significance, regularization, terms, renormal...",Particle physics or high-energy physics is the...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D,"[(Which, PRON), (of, ADP), (the, DET), (follow...","[statements, relationship, dimensions, object,...","A metamaterial (from the Greek word μετά meta,..."


In [96]:
# check for empty list in propn_noun
# train[train['propn_noun'].map(lambda x: len(x)==0)]

In [97]:
train.to_csv('data/trainWithWiki.csv', index=False)